<a href="https://colab.research.google.com/github/basharE/CVDs_Predictors/blob/main/CVDs_Predictors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing packages/Imports

In [84]:
!pip install -U -q PyDrive
# !pip install shap
# !pip install boruta
# !pip install BorutaShap
# !pip install catboost
# !pip install verstack

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as R2
from scipy.special import softmax
# from boruta import BorutaPy
# from BorutaShap import BorutaShap
# from verstack import LGBMTuner
from sklearn.model_selection import cross_val_score

import pandas as pd
import numpy as np
import seaborn as sn
sn.set_style(style='darkgrid')
import matplotlib.pyplot as plt
import math

# Uploading data 

## Connecting to google drive

In [85]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Uploading csv files

In [86]:
df_Cleveland = pd.read_csv("/content/drive/MyDrive/CVD_DATA/cleveland.data", delimiter=" ")
df_Hungarian = pd.read_csv("/content/drive/MyDrive/CVD_DATA/hungarian.data", delimiter=" ")
df_Switzerland = pd.read_csv("/content/drive/MyDrive/CVD_DATA/switzerland.data", delimiter=" ")
df_LongBeachVA = pd.read_csv("/content/drive/MyDrive/CVD_DATA/long-beach-va.data", delimiter=" ")

In [87]:
df = pd.concat([df_Cleveland, df_Hungarian, df_Switzerland, df_LongBeachVA])

In [88]:
df_co = df[['smoke'
,'cigs'
,'years'
,'age'      
,'sex'       
,'cp'        
,'trestbps'
,'chol'   
,'fbs'       
,'restecg'
,'thalach'   
,'exang'    
,'oldpeak'
,'slope'    
,'ca'      
,'thal'
,'num']]

In [89]:
conditions = [
    (df_co['smoke'] == -9) & (df_co['cigs'] == -9) & (df_co['years'] == -9),
    (df_co['smoke'] == 0) | (df_co['cigs'] == 0) | (df_co['years'] == 0),
    (df_co['smoke'] > 0) | (df_co['cigs'] > 0) | (df_co['years'] > 0)
]

values = [-9, 0, 1]

df_co['is_smoker'] = np.select(conditions, values)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [90]:
conditions = [
    (df_co['num'] > 0),
    (df_co['num'] == 0)
]

values = [1, 0]

df_co['num_new'] = np.select(conditions, values)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Correlation

In [91]:
df_smoker = df_co[['is_smoker', 'num', 'num_new']]

In [92]:
df_smoker = df_smoker[df_smoker['is_smoker'] != -9]

In [93]:
#########################################################
# Cross tabulation between is_smoker and num
CrosstabResult=pd.crosstab(index=df_smoker['num'],columns=df_smoker['is_smoker'])
print(CrosstabResult)
 
# importing the required function
from scipy.stats import chi2_contingency
 
# Performing Chi-sq test
ChiSqResult = chi2_contingency(CrosstabResult)
 
# P-Value is the Probability of H0 being True
# If P-Value&gt;0.05 then only we Accept the assumption(H0)
 
print('The P-Value of the ChiSq Test is:', ChiSqResult[1])

is_smoker    0    1
num                
0          106  109
1           47   70
2           32   46
3           37   42
4            4   19
The P-Value of the ChiSq Test is: 0.03588940652959348


In [94]:
#########################################################
# Cross tabulation between is_smoker and num
CrosstabResult=pd.crosstab(index=df_smoker['is_smoker'],columns=df_smoker['num_new'])
print(CrosstabResult)
 
# importing the required function
from scipy.stats import chi2_contingency
 
# Performing Chi-sq test
ChiSqResult = chi2_contingency(CrosstabResult)
 
# P-Value is the Probability of H0 being True
# If P-Value&gt;0.05 then only we Accept the assumption(H0)
 
print('The P-Value of the ChiSq Test is:', ChiSqResult[1])

num_new      0    1
is_smoker          
0          106  120
1          109  177
The P-Value of the ChiSq Test is: 0.05599361583364363


## Feature Importance


In [95]:
df_co

,smoke,cigs,years,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,is_smoker,num_new
0,-9,50,20,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0,1,0
1,-9,40,40,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2,1,1
2,-9,20,35,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1,1,1
3,-9,0,0,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0,0,0
4,-9,0,0,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,54,0,4,127,333,1,1,154,0,0.0,-9,-9,-9,1,0,1
196,1,15,30,62,1,1,-9,139,0,1,-9,-9,-9.0,-9,-9,-9,0,1,0
197,1,20,40,55,1,4,122,223,1,1,100,0,0.0,-9,-9,6,2,1,1
198,0,10,20,58,1,4,-9,385,1,2,-9,-9,-9.0,-9,-9,-9,0,0,0


In [96]:
columns_to_delete = ['smoke', 'cigs', 'years', 'num_new']
df_co = df_co.drop(columns_to_delete, axis=1)

In [99]:
df_co = df_co[df_co['age'] != -9]
df_co = df_co[df_co['sex'] != -9]
df_co = df_co[df_co['cp'] != -9]
df_co = df_co[df_co['trestbps'] != -9]
df_co = df_co[df_co['chol'] != -9]
df_co = df_co[df_co['fbs'] != -9]
df_co = df_co[df_co['restecg'] != -9]
df_co = df_co[df_co['thalach'] != -9]
df_co = df_co[df_co['exang'] != -9]
df_co = df_co[df_co['oldpeak'] != -9]
df_co = df_co[df_co['slope'] != -9]
df_co = df_co[df_co['ca'] != -9]
df_co = df_co[df_co['thal'] != -9]
df_co = df_co[df_co['num'] != -9]
df_co = df_co[df_co['is_smoker'] != -9]

In [100]:
df_co

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,is_smoker
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0,1
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,57,1,2,154,232,0,2,164,0,0.0,1,1,3,1,1
279,58,0,4,130,197,0,0,131,0,0.6,2,0,3,0,0
280,57,1,4,110,335,0,0,143,1,3.0,2,1,7,2,1
281,47,1,3,130,253,0,0,179,0,0.0,1,0,3,0,0
